In [2]:
import bs4 as bs 
import requests
import json
import csv
import re

source_url="tiki.vn"
search_page_url = "https://tiki.vn/search?q=genshin"
api_url = "https://tiki.vn/api/v2/products/"
user_agent={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20120101 Firefox/33.0'}

## class product_list_container

In [3]:
sauce=requests.get(search_page_url,headers=user_agent)
soup=bs.BeautifulSoup(sauce.text,'html.parser')

In [8]:
loop_to_end=soup.find('div',class_="Pagination__Root-sc-cyke21-0 gNgpAR")
end_of_page=int(re.findall('[0-9]+',[loop_to_end.find_all('li')[link].find('a').get('href') for link in range(0,len(loop_to_end.find_all('li')))][-2])[0])


In [10]:
header=['id','sku','name','short_url','short_description','price','original','rating_average','review_count','productset_group_name','day_ago_created','all_time_quality_sold','image_url']
with open("Tiki_crawl.csv" , "w", newline = '',encoding='utf-8') as file_output:
        writer = csv.DictWriter(file_output,delimiter =',', lineterminator = '\n', fieldnames = header)
        writer.writeheader()
        #Move to next page
        for x in range(0,end_of_page):
            next_page=search_page_url+f'&page={x}'
            # Get 40 urls product
            list_product=soup.find_all(class_='product-item')
            # Get link
            product_url=[source_url+list_product[link].get('href') for link in range(0,len(list_product))]
            #Get list_id
            list_id=[product_url[id_product].split('.html')[0].split('-')[-1][1:] for id_product in range(0,len(list_product))]
            
            for i in range(0,len(list_id)):
                try:
                    json_url=api_url+list_id[i]
                    sauce_json=requests.get(json_url,headers=user_agent)
                    soup_json=bs.BeautifulSoup(sauce_json.text,"html.parser")
                    data=list(json.loads(soup_json.string).items())
                    id=data[0][1] #ID 
                    sku=data[2][1]#SKU
                    name=data[3][1]#Name
                    short_url=data[6][1]  #short_url
                    short_description=data[9][1] #short_description
                    price=data[10][1] #price
                    original=data[12][1]#original price
                    rating_average=data[17][1]#rating_average
                    review_count=data[18][1] #review_count
                    productset_group_name=data[25][1]  #productset_group_name
                    day_ago_created=data[33][1]#day_ago_created
                    all_time_quality_sold=data[34][1]#all_time_quality_sold
                    image_url=data[41][1]#image_url
                    writer.writerow ({header[0]:id,header[1]:sku,header[2]:name, header[3]:short_url, header[4]:short_description, header[5]: price,header[6]: original, header[7]:rating_average ,header[8]: review_count,header[9]:productset_group_name,header[10]:day_ago_created,header[11]:all_time_quality_sold,header[12]:image_url})
                except: TypeError